### Overview of the Dataset
The dataset contains aggregate individual statistics for 67 NBA seasons. It includes basic box-score attributes such as points, assists, rebounds etc. The data was originally scraped from basketball reference.

The dataset includes 24691 observations across 51 columns.

https://www.kaggle.com/drgilermo/nba-players-stats/data

In [12]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd")

In [13]:
player_stats = pd.read_csv('Seasons_Stats.csv')

In [14]:
player_stats.head(5)

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.000,Curly Armstrong,G-F,31.000,FTW,63.000,nan,nan,nan,...,0.705,nan,nan,nan,176.000,nan,nan,nan,217.000,458.000
1,1,1950.000,Cliff Barker,SG,29.000,INO,49.000,nan,nan,nan,...,0.708,nan,nan,nan,109.000,nan,nan,nan,99.000,279.000
2,2,1950.000,Leo Barnhorst,SF,25.000,CHS,67.000,nan,nan,nan,...,0.698,nan,nan,nan,140.000,nan,nan,nan,192.000,438.000
3,3,1950.000,Ed Bartels,F,24.000,TOT,15.000,nan,nan,nan,...,0.559,nan,nan,nan,20.000,nan,nan,nan,29.000,63.000
4,4,1950.000,Ed Bartels,F,24.000,DNN,13.000,nan,nan,nan,...,0.548,nan,nan,nan,20.000,nan,nan,nan,27.000,59.000


In [15]:
player_stats.drop(columns=['Unnamed: 0', 'Year'], inplace=True)

In [16]:
player_stats.describe()

,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,24616.000,24624.000,18233.000,24138.000,24101.000,24538.000,18839.000,24525.000,20792.000,20792.000,...,23766.000,20797.000,20797.000,24312.000,24624.000,20797.000,20797.000,19645.000,24624.000,24624.000
mean,26.664,50.837,23.593,1209.720,12.479,0.493,0.159,0.325,6.182,13.709,...,0.719,62.189,147.199,224.637,114.853,39.897,24.470,73.940,116.339,510.116
std,3.842,26.496,28.632,941.147,6.039,0.094,0.187,0.219,4.873,6.636,...,0.142,67.325,145.922,228.190,135.864,38.713,36.935,67.714,84.792,492.923
min,18.000,1.000,0.000,0.000,-90.600,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,24.000,27.000,0.000,340.000,9.800,0.458,0.005,0.208,2.600,8.800,...,0.657,12.000,33.000,51.000,19.000,9.000,3.000,18.000,39.000,106.000
50%,26.000,58.000,8.000,1053.000,12.700,0.506,0.064,0.296,5.400,12.700,...,0.743,38.000,106.000,159.000,68.000,29.000,11.000,55.000,109.000,364.000
75%,29.000,75.000,45.000,1971.000,15.600,0.544,0.288,0.400,9.000,18.100,...,0.808,91.000,212.000,322.000,160.000,60.000,29.000,112.000,182.000,778.000
max,44.000,88.000,83.000,3882.000,129.100,1.136,1.000,6.000,100.000,100.000,...,1.000,587.000,1111.000,2149.000,1164.000,301.000,456.000,464.000,386.000,4029.000


In [17]:
player_stats.shape

(24691, 51)

In [18]:
player_stats.dtypes

Player     object
Pos        object
Age       float64
Tm         object
G         float64
GS        float64
MP        float64
PER       float64
TS%       float64
3PAr      float64
FTr       float64
ORB%      float64
DRB%      float64
TRB%      float64
AST%      float64
STL%      float64
BLK%      float64
TOV%      float64
USG%      float64
blanl     float64
OWS       float64
DWS       float64
WS        float64
WS/48     float64
blank2    float64
OBPM      float64
DBPM      float64
BPM       float64
VORP      float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
dtype: object

In [19]:
missing_values_count = player_stats.isnull().sum()
print(missing_values_count)

Player       67
Pos          67
Age          75
Tm           67
G            67
GS         6458
MP          553
PER         590
TS%         153
3PAr       5852
FTr         166
ORB%       3899
DRB%       3899
TRB%       3120
AST%       2136
STL%       3899
BLK%       3899
TOV%       5109
USG%       5051
blanl     24691
OWS         106
DWS         106
WS          106
WS/48       590
blank2    24691
OBPM       3894
DBPM       3894
BPM        3894
VORP       3894
FG           67
FGA          67
FG%         166
3P         5764
3PA        5764
3P%        9275
2P           67
2PA          67
2P%         195
eFG%        166
FT           67
FTA          67
FT%         925
ORB        3894
DRB        3894
TRB         379
AST          67
STL        3894
BLK        3894
TOV        5046
PF           67
PTS          67
dtype: int64
